In [40]:
### This notebook is the analysis of BCL-2 mutations, permutation analysis

import pandas as pd
import os
import pickle
from Bio import SeqIO
from Bio.Seq import Seq
from utils import *
import pickle
import re

In [2]:
dir_out = './figure6/'

***1. BCL-2 mutation information(input for R lollipop plot and permutation test)***

In [3]:
# Read the BCL-2 mutation maf files
dir_maf = '../maf_out/maf_cohorts_060121'
feature = 'histology'
feat = 'Lymph-BNHL.csv'

df_mut = pd.read_csv(os.path.join(dir_maf, feature,feat), sep = '\t')

/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-bzb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (1,17,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# Filter out BCL-2 silent mutation
df_bcl2_syn = df_mut[(df_mut['Variant_Classification'] == 'Silent') & (df_mut['Hugo_Symbol'] == 'BCL2')]

# Save the synonymous mutation dataframe for permutation and lolliplot input
#df_bcl2_syn.to_csv(os.path.join(dir_out,'data','df_bcl2_syn.csv'))

In [16]:
len(df_bcl2_syn) ### The number of synonymous mutations in total

41

In [20]:
len(df_bcl2_syn['Donor_ID'].unique()) ### Total number of patients

26

In [ ]:
# bcl2_syn_mut = df_bcl2_syn['Start_position'].to_list() # Don't know what this is for

***2. BCL-2 CDS sequence(permutation test, sequence is copy and pasted into R)***

In [9]:
# Load annotation data
dict_name = pickle.load(open('../anno_ref/proc_refs/dict_name_062121.pkl','rb'))
dict_transcript_info = pickle.load(open('../anno_ref/proc_refs/dict_transcript_info_062121.pkl','rb'))
dict_record = SeqIO.to_dict(SeqIO.parse('../anno_ref/gencode_v19/GRCh37.p13.genome.fa', 'fasta'))

In [10]:
# This information was mannually got from Uniprot
BH4 = (27,90); BH3 = (276,321); BH1 = (405,465);BH2 = (558,606)

In [11]:
# The canonical transcript is ENST00000398117
bcl2_info = dict_transcript_info['ENST00000398117']

In [12]:
### Get the cds sequence of BCL2
list_cds = get_mrna_position('ENST00000398117', dict_transcript_info, '-')
seq_transcript = get_transcript_sequence('ENST00000398117', dict_transcript_info, dict_record, '-')
seq_cds = get_cdna_sequence(list_cds, '-', seq_transcript)

In [51]:
### how many patients have BH4 mutation

# BH4 position
BH4_start = 60985899-27+1
BH4_end = 60985899-90+1

# Print out Patient ID that have BH4 mutation
dlist = []
for i in df_bcl2_syn.index:
    pos = re.findall('\d+',df_bcl2_syn.loc[i,'Genome_Change'])[1]
    pos_list.append(pos)
    if int(pos) >= BH4_end and int(pos) <=BH4_start:
        print(df_bcl2_syn.loc[i,'Donor_ID'])
        dlist.append(df_bcl2_syn.loc[i,'Donor_ID'])

len(set(dlist))

DO52685
DO52695
DO27857
DO52652
DO27859
DO27809
DO52664
DO52689
DO52669


9

***3. The data goes into R***
06-BCL-2_enrichment.R for the permutation plot
